# Save, Load and Export Models in Keras

## Task 1: Import Libraries

In [1]:
import tensorflow as tf
import numpy as np
import os

print('This notebook works with TensorFlow version:', tf.__version__)

folders = ['tmp', 'models', 'model_name', 'weights']
for folder in folders:
    if not os.path.isdir(folder):
        os.mkdir(folder)

print(os.listdir('.'))

This notebook works with TensorFlow version: 2.0.1
['.ipynb_checkpoints', 'models', 'model_name', 'Student Notebook.ipynb', 'tmp', 'weights']


## Task 2: Create Model

In [2]:
## create model architechture, compile and summarize.
def create_model():
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units=128, input_shape=(784,), activation='relu'),
        tf.keras.layers.Dense(units=128, activation='relu'),
        tf.keras.layers.Dense(units=10, activation='softmax')
    ])
    
    model.compile( loss = 'categorical_crossentropy',
                    optimizer = 'adam',
                    metrics = ['accuracy']
                  )
    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


## Task 3: Data Preprocessing

In [3]:
## import fashion-mnist data from keras.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

## reshape and normalize the data.
x_train = np.reshape(x_train, (x_train.shape[0], 784 ))/255.
x_test = np.reshape(x_test, (x_test.shape[0], 784 ))/255.

#one-hot encoding labels.
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

## Task 4: Model Checkpoint During Training

In [4]:
## save model weights
checkpoint_dir = 'weights/'
_ = model.fit(x_train, y_train,
            validation_data=(x_test, y_test),
            epochs=2,
            batch_size=512,
            callbacks = [
                tf.keras.callbacks.ModelCheckpoint(
                                                os.path.join(checkpoint_dir, 'epoch_{epoch:02d}_accuracy_{val_accuracy:.4f}'),
                                                monitor='val_accuracy',
                                                save_best_only=True,
                                                save_weights_only=True
                                                )
                        ]
             )

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 5s 86us/sample - loss: 0.6838 - accuracy: 0.7702 - val_loss: 0.4988 - val_accuracy: 0.8242
Epoch 2/2
60000/60000 [==============================] - 4s 69us/sample - loss: 0.4276 - accuracy: 0.8510 - val_loss: 0.4314 - val_accuracy: 0.8459


In [5]:
os.listdir(checkpoint_dir)

['checkpoint',
 'epoch_01_accuracy_0.8242.data-00000-of-00001',
 'epoch_01_accuracy_0.8242.index',
 'epoch_01_accuracy_0.8247.data-00000-of-00001',
 'epoch_01_accuracy_0.8247.index',
 'epoch_01_accuracy_0.8341.data-00000-of-00001',
 'epoch_01_accuracy_0.8341.index',
 'epoch_02_accuracy_0.8348.data-00000-of-00001',
 'epoch_02_accuracy_0.8348.index',
 'epoch_02_accuracy_0.8436.data-00000-of-00001',
 'epoch_02_accuracy_0.8436.index',
 'epoch_02_accuracy_0.8459.data-00000-of-00001',
 'epoch_02_accuracy_0.8459.index']

## Task 5: Load Weights

In [6]:
model = create_model()
print(model.evaluate(x_test, y_test, verbose=False))

[2.374910878753662, 0.0955]


In [8]:
model.load_weights('weights/epoch_02_accuracy_0.8348')
print(model.evaluate(x_test, y_test, verbose=False))

[0.4229256177186966, 0.85005]


## Task 6: Saving Complete Model During Training

In [9]:
## save model weights
models_dir = 'models/'

# create model , so we don't use pre-trained model.
model = create_model()

_ = model.fit(x_train, y_train,
            validation_data=(x_test, y_test),
            epochs=2,
            batch_size=512,
            callbacks = [
                tf.keras.callbacks.ModelCheckpoint(
                                                os.path.join(models_dir, 'epoch_{epoch:02d}_accuracy_{val_accuracy:.4f}.h5'),
                                                monitor='val_accuracy',
                                                save_best_only=False,
                                                save_weights_only=False
                                                )
                        ]
             )

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 3s 42us/sample - loss: 0.6925 - accuracy: 0.7684 - val_loss: 0.5045 - val_accuracy: 0.8204
Epoch 2/2
60000/60000 [==============================] - 2s 34us/sample - loss: 0.4343 - accuracy: 0.8483 - val_loss: 0.4442 - val_accuracy: 0.8402


In [10]:
os.listdir(models_dir)

['epoch_01_accuracy_0.8204.h5', 'epoch_02_accuracy_0.8402.h5']

## Task 7: Load Models

In [11]:
model = create_model()
print(model.evaluate(x_test, y_test, verbose=False))

[2.4871178932189943, 0.0722]


In [13]:
model = tf.keras.models.load_model('models/epoch_02_accuracy_0.8402.h5')
print(model.evaluate(x_test, y_test, verbose=False))
model.summary()

[0.4442192112922668, 0.8402]
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_7 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


## Task 8: Manually Saving Weights and Models

In [14]:
## save model weights
model.save_weights('temp/manually_saved.w')
os.listdir('temp')

['checkpoint',
 'manually_saved.w.data-00000-of-00001',
 'manually_saved.w.index']

In [15]:
## save entire model
model.save('temp/manually_saved.h5')
os.listdir('temp')

['checkpoint',
 'manually_saved.h5',
 'manually_saved.w.data-00000-of-00001',
 'manually_saved.w.index']

## Task 9: Exporting and Restoring SavedModel Format

In [16]:
## if path to model.save('path') is directory(not a file path) it will save the model in that directory.
model.save('model_name')
os.listdir('model_name')

W0331 22:52:04.464663  4168 deprecation.py:506] From c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


['assets', 'saved_model.pb', 'variables']

In [18]:
## what if you have saved model and want to bring it as keras model !
## you give dir path of your model to load_model(path).
model = tf.keras.models.load_model('model_name')
model.evaluate(x_test, y_test, verbose=False)

[0.44421920738220216, 0.8402]